In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
from typing import List, Dict, Tuple

vector_db_path = "vectorstores/db_faiss"

# Initialize the embedding model
embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")

# Load the FAISS vector database with the embedding model
db = FAISS.load_local(folder_path=vector_db_path, embeddings=embedding_model, allow_dangerous_deserialization = True)

# Perform a similarity search
def search_query(query: str, k: int = 30):
    """
    Perform a similarity search on the vector database.
    
    :param query: The query string.
    :param k: The number of top results to return.
    :return: The top results as a list of strings.
    """
    results = db.similarity_search(query, k=k)
    return [result.page_content for result in results]

# Example query
query = "Thủ tục cấp chứng chỉ hành nghề dược gồm các nội dung gì?"
top_results = search_query(query, k=30)

# Print results
print("\nTop Results:")
for result in top_results:
    print(result)



Top Results:
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 7. Thủ tục thu hồi Chứng chỉ hành nghề dược > 3. Trách nhiệm của cơ quan cấp, thu hồi Chứng chỉ hành nghề dược:: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược > 10. Mẫu Chứng chỉ hành nghề dược:: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 4. Quy định chi tiết về hồ sơ đề nghị cấp lại Chứng chỉ hành nghề dược > 1. Hồ sơ đề nghị cấp lại Chứng chỉ hành nghề dược thực hiện theo quy định tại Điều 25 của Luật dược và được quy định cụ thể như sau:: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 3. Quy định chi tiết về hồ sơ đề nghị cấp Chứng chỉ hành nghề dược > 1. Hồ sơ đề nghị cấp Chứng chỉ hành nghề dược thực hiện theo quy định tại Điều 24 của Luật dược và được quy định cụ thể như sau:: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược > 11. Bộ trưởng Bộ Y tế quy định t

Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [2]:
query = "Thủ tục cấp chứng chỉ hành nghề dược gồm các nội dung gì?"
top_results = search_query(query, k=30)

# Print results
print("\nTop Results:")
for result in top_results:
    print(result)



Top Results:
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 7. Thủ tục thu hồi Chứng chỉ hành nghề dược > 3. Trách nhiệm của cơ quan cấp, thu hồi Chứng chỉ hành nghề dược:: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược > 10. Mẫu Chứng chỉ hành nghề dược:: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 4. Quy định chi tiết về hồ sơ đề nghị cấp lại Chứng chỉ hành nghề dược > 1. Hồ sơ đề nghị cấp lại Chứng chỉ hành nghề dược thực hiện theo quy định tại Điều 25 của Luật dược và được quy định cụ thể như sau:: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 3. Quy định chi tiết về hồ sơ đề nghị cấp Chứng chỉ hành nghề dược > 1. Hồ sơ đề nghị cấp Chứng chỉ hành nghề dược thực hiện theo quy định tại Điều 24 của Luật dược và được quy định cụ thể như sau:: 
Chương II CHỨNG CHỈ HÀNH NGHỀ DƯỢC > Điều 6. Quy định chi tiết về thủ tục cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược > 11. Bộ trưởng Bộ Y tế quy định t

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import BitsAndBytesConfig

import torch

# Define the path for the model
model_file = "AITeamVN/Vi-Qwen2-7B-RAG"
# model_file = "vilm/vinallama-7b-chat"

# Define the FAISS vector store path
vector_db_path = "vectorstores/db_faiss"

# Load the Hugging Face model and tokenizer
def load_huggingface_model(model_file):
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Tải trọng số được lượng hóa trước theo định dạng 4 bit
        bnb_4bit_quant_type="nf4",  # Sử dụng loại lượng hóa "nf4" cho trọng số 4 bit
        bnb_4bit_compute_dtype=torch.bfloat16,  # Sử dụng torch.bfloat16 cho các phép tính trung gian
        bnb_4bit_use_double_quant=True,  # Sử dụng độ chính xác kép để lượng hóa kích hoạt
    )
    # quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold = 6.0)
    model = AutoModelForCausalLM.from_pretrained(model_file, device_map="auto", quantization_config=quantization_config,)
    tokenizer = AutoTokenizer.from_pretrained(model_file)
    return model, tokenizer

# Read the vector database (FAISS)
def read_vectors_db():
    embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
    return db

# Perform similarity search on the vector database
def search_vector_db(query, k=2):
    db = read_vectors_db()
    results = db.similarity_search(query, k=k)
    return [result.page_content for result in results]
# Load the model and tokenizer
model, tokenizer = load_huggingface_model(model_file)


system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể."
template = '''Chú ý các yêu cầu sau:
- Câu trả lời phải chính xác và đầy đủ nếu ngữ cảnh có câu trả lời. 
- Chỉ sử dụng các thông tin có trong ngữ cảnh được cung cấp.
- Chỉ cần từ chối trả lời và không suy luận gì thêm nếu ngữ cảnh không có câu trả lời.
Hãy trả lời câu hỏi dựa trên ngữ cảnh:
### Ngữ cảnh :
{context}

### Câu hỏi :
{question}

### Trả lời :'''





/home/phongnd/anaconda3/envs/llm2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.18s/it]


In [4]:
question = '''Hồ sơ đề nghị cấp chứng chỉ hành nghề  dược đối với người nước ngoài gồm những nội dung nào, trả lời một cách chi tiết và đầy đủ nhất'''
context_list = search_vector_db(question, k = 50)
context = "\n".join(context_list)
conversation = [{"role": "system", "content": system_prompt }]
conversation.append({"role": "user", "content": template.format(context = context, question = question)})
with torch.inference_mode():
    text = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True)
    model_inputs = tokenizer(text,return_tensors="pt").to(model.device)
    attention_mask = model_inputs["attention_mask"]
    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=2048,
        temperature = 0.5,
        #top_p=0.95,
        #top_k=40,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [5]:
import textwrap

# Example text (you can replace this with your query result)
text = response

# Function to wrap text
def wrap_text(text, width=80):
    wrapper = textwrap.TextWrapper(width=width, replace_whitespace=False)
    wrapped_text = "\n".join([wrapper.fill(line) for line in text.splitlines()])
    return wrapped_text                                                                                                                                                                                            

# Wrap and print the text
wrapped_text = wrap_text(text, width=140)
print(wrapped_text)


Dựa trên ngữ cảnh được cung cấp, hồ sơ đề nghị cấp Chứng chỉ hành nghề dược đối với người nước ngoài bao gồm các nội dung sau:

1. **Giấy tờ do cơ quan có thẩm quyền nước ngoài cấp**:
   - Các giấy tờ này phải được hợp pháp hóa lãnh sự theo quy định.
   - Các giấy tờ này phải có bản dịch sang tiếng Việt và được công chứng theo quy định.

2. **Số lượng hồ sơ**:
   - Hồ sơ đề nghị cấp Chứng chỉ hành nghề dược thực hiện theo quy định tại Điều 24 của Luật dược và được quy định cụ thể như sau:
     - **Điều 3, Khoản 3**: Số lượng hồ sơ quy định tại Điều này là 01 bộ.

3. **Thủ tục nộp hồ sơ**:
   - Người đề nghị cấp Chứng chỉ hành nghề dược nộp hồ sơ trực tiếp hoặc gửi qua đường bưu điện về cơ quan có thẩm quyền.

4. **Thời hạn xử lý**:
   - Trong thời hạn 05 ngày làm việc, kể từ ngày cấp, cấp lại, điều chỉnh nội dung Chứng chỉ hành nghề dược, cơ quan tiếp nhận hồ sơ công
bố, cập nhật trên Cổng thông tin điện tử của đơn vị.

5. **Trách nhiệm của cơ quan cấp Chứng chỉ**:
   - Cơ quan cấp Chứ